# Frontier extraction using python

Frontier extraction is a way of getting the contour of an object on a scene. It is really useful and the idea behind it is very easy, in this notebook I'll go trough a simple example using the MNIST handwritten digits.

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from skimage.morphology import erosion, diamond

path_to_data = '../input/digit-recognizer/train.csv'
data = pd.read_csv(path_to_data)

del data['label']

In [ ]:
rows = 4
columns = 3

fig, ax = plt.subplots(rows, columns, figsize = (12, 14))

for i in range(rows):
    for j in range(columns):
        index = np.random.randint(data.shape[0])

        image = np.asarray(data.iloc[index][:]).reshape((28, 28))
        ax[i][j].imshow(image, cmap = 'gray')

plt.tight_layout()
plt.show()

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.describe()

We can see from the past outputs that the values that range from 0 to 255, as each pixel is stored in a 8 bit value. To apply mathematical morphology we need this values to be binary, 0 and 1. So the first thing we need to do is binarize the images.  
We'll do this by the simplest method there is, tresholding. In order to perform threshold, we are going to define a *t* value and then check each value to compare it to *t*, if the value is greater than *t* we set the binary value as 1, and 0 otherwise.

In [ ]:
def to_binary(image, treshold):
    binary = np.zeros(image.shape)
    for i in range(784):
        if(image[i] > treshold):
            binary[i] = 1
    
    return binary

Let's create a new dataset of binary images.

In [ ]:
data = np.asarray(data)
data_binary = data.copy()
for i in range(data.shape[0]):
    data_binary[i] = to_binary(data[i], 0)

Try changing the threshold value and see what happens!

In [ ]:
pd.DataFrame(data = data_binary).describe()

Reshaping the arrays to make them easier to handle.

In [ ]:
data = data.reshape(-1, 28, 28)
data_binary = data_binary.reshape(-1, 28, 28)

Now we do the frontier extraction. The key to perform this is to consider the images as sets and not as arrays. The next step is to perform an erosion, a morphological operation that has the effect of *"making smaller"* the objects in the scene, as it can be seen in the example below.  

![Erosion](https://homepages.inf.ed.ac.uk/rbf/HIPR2/figs/erodbin.gif)  

This is acheived by passing a structuring element through the scene. A structuring element is an array, usually of small dimensions (for example, 3x3), which has binary values and a defined center. When all the values in the array coincide with the values in the scene the only value you leave in the resulting scene is the center of the structuring element. To make this clear here is an example of performing erosion on a 2 with a cross structuring element.

![Erosion example se](https://www.researchgate.net/publication/305375221/figure/fig1/AS:393507714945026@1470830958326/The-erosion-of-an-object-by-a-structuring-element.png)

Now that we know what erosion is the idea comes quickly, if you perform a (set) difference between the original image and the eroded image we can easily see that the result is going to be the frontier of the original image. Let's not show examples yet, let's implement it and check wether this is true or not.  

This is easy to implement and easier now that skimage gives us all the tools we need.

In [ ]:
test = data_binary[0,:,:]
test_eroded = erosion(test, diamond(1))
frontier = np.copy(test)

for i in range(28):
    for j in range(28):
        if(test[i,j] != test_eroded[i,j]):
            frontier[i,j] = 1
        
        else:
            frontier[i,j] = 0
            
fig, ax = plt.subplots(1, 3, figsize = (20, 10))
ax[0].imshow(test, cmap = 'binary')
ax[0].set_title("Orignal")

ax[1].imshow(test_eroded, cmap = 'binary')
ax[1].set_title("Eroded")

ax[2].imshow(frontier, cmap = 'binary')
ax[2].set_title("Frontier (Original - Eroded)")

plt.show()


We can see that the reasoning was right! The eroded image does look smaller than the original and the result by the (set) difference does show the frontier of the original image. One thing is that you should really choose your structuring element wisely, some are going to work on certain scenes and others are just not going to work as we want. Check out all the [structuring elements](https://scikit-image.org/docs/dev/api/skimage.morphology.html) that skimage has (and why not, try to create your own and test them!).  

Finally, let's define a function so we can use it in a cleaner way.

In [ ]:
def extract_frontier(image):
    test = image.copy()
    test_eroded = erosion(test, diamond(1))
    frontier = np.copy(test)

    for i in range(28):
        for j in range(28):
            if(test[i,j] != test_eroded[i,j]):
                frontier[i,j] = 1

            else:
                frontier[i,j] = 0
    
    return frontier

In [ ]:
rows = 4
columns = 3

fig, ax = plt.subplots(rows, columns, figsize = (12, 14))

for i in range(rows):
    for j in range(columns):
        index = np.random.randint(data_binary.shape[0])

        ax[i][j].imshow(extract_frontier(data_binary[index]), cmap = 'binary')

plt.tight_layout()
plt.show()